# Data Fusion Contest 2026 - Задача 1 "Страж"

Участникам необходимо решить задачу классификации неподтвержденных операций клиентов банка на основе данных историй операций в различных каналах.

## Постановка задачи

Необходимо решить задачу классификации для операций, которые по различным причинам, связанным с безопасностью, были не подтверждены клиентами банка. То есть, участникам предстоит разработать алгоритм, реализующий ядро системы антифрода для предотвращения мошеннических операций.

Для обучения моделей предоставляются операции 100,000 клиентов за 1.5 года, организованные по времени на 4 отдельные периода:

1. **Pre-train**, с `2023-10-01` по `2024-09-30`. История операций клиентов, которую можно использовать для предобучения и извлечения признаков. Не имеет разметки по построению.
2. **Train**, с `2024-10-01` по `2025-05-31`. История операций клиентов, в которой у них появляются нежелательные операции. Имеет разметку как неподтвержденных операций (🔴 целевой класс, «красный свет»), так и подозрительных операций, подтвержденных клиентами (🟡 «желтый свет», не является целевым классом). Все остальные операции без обратной связи клиента стоит считать подтвержденными (🟢 «зеленый свет»).
3. **Pre-test**, с `2025-06-01` по `2025-08-09`. История операций клиентов, которую можно использовать для классификации самих тестовых меток. Не имеет разметки, так как является частью тестовых данных.
4. **Test**, с `2025-06-01` по `2025-08-09`. Заключительный день истории операций клиентов, каждую из которых требуется классифицировать и отправить в рамках соревнования. У каждого клиента этот заключительный день выбран случайно.

У задачи имеется ряд особенностей для исследователей и практиков анализа данных:

- Целевой класс очень редкий: в тренировочных данных всего 51 тысяча неподтвержденных операций (🔴) и 36 тысяч подтвержденных (🟡).
- Алгоритм должен работать в приближенной к реальной постановке, с учетом времени. Как если бы классификатор работал здесь и сейчас, а не размечал уже ранее случившиеся инциденты пост-фактум.
- Суммарный объем данных в задаче превышает 200 миллионов операций. Подробнее про структуру данных можно узнать на странице «Данные».

## Формат решений

Это табличное соревнование с разметкой предоставленного вам `.csv` файла. Вам необходимо создать алгоритм, способный по предоставленным в рамках соревнования данным, создать новый табличный `.csv` файл с двумя столбцами:

```text
event_id,predict
125854726334416,-0.338988
125949211749418,-4.100378
...
124738035029214,0.004335
```

- `event_id` — идентификатор операции;
- `predict` — предсказание вашего алгоритма.

Предсказания необходимо построить для всех 633,683 операций тестовых данных.

Пример `sample_submit.csv` доступен на странице «Данные».

## Проверка решений

Решения проверяются автоматически путем сопоставления с известной истинной разметкой статусов операций по классу неподтвержденных операций (🔴) в тестовый период данных. Истинная разметка тестовых данных доступна только организаторам.

Метрика соревнования — **PR-AUC** (Area Under Precision-Recall Curve; площадь под кривой «полнота-точность») по операциям.

Для расчета метрики используется её `sklearn` имплементация:

```python
from sklearn.metrics import average_precision_score
```

Обратите внимание, что использование функции `auc` для самой Precision-Recall кривой средствами sklearn приведет к завышению показателей метрики ввиду особенностей процедуры подсчета (интерполяции точек кривой).

Тестовые данные представляют собой 70 дней операций, сгруппированные в 10 недель.
Отсчет идет с `2025-06-01` по `2025-08-09`, первая неделя идет с `2025-06-01` по `2025-06-07`, и т.д.

Соотношение public/private в соревновании составляет 30/70 и разделено по времени (по неделям):

- Недели 1, 3 и 5 используются для результатов на public лидерборде.
- Остальные 7 недель используются для результатов на private лидерборде.
- Победители соревнования определяются по результатам на private лидерборде.
- Для private лидерборда можно выбрать до 2-ух финальных решений.

## Данные

Для решения задачи 1 «Страж» предлагается несколько наборов данных и материалов. Данные разделены на 4 временных периода:

1. **Pre-train**, с `2023-10-01` по `2024-09-30`. Только данные самих операций.
2. **Train**, с `2024-10-01` по `2025-05-31`. И данные операций, и разметка целевой переменной к ним.
3. **Pre-test**, с `2025-06-01` по `2025-08-09`. Начало тестовых данных с историей операций, но без разметки к ним.
4. **Test**, с `2025-06-01` по `2025-08-09`. Финальный день тестовых данных операций клиентов, по которому требуется построить разметку и отправить ее в качестве решения.

Для удобства участников, **pre-train** и **train** данные разделены на 3 части. Каждый файл содержит свою группу клиентов на основе их `customer_id`. Сами записи отсортированы по клиентам `customer_id` и по времени произведения операций `event_dttm`.

## Данные операций за Pre-train и Train

- `pretrain_part_1.parquet` — 625.2MB, операции первой трети клиентов с `2023-10-01` по `2024-09-30`
- `pretrain_part_2.parquet` — 622.8MB, операции второй трети клиентов с `2023-10-01` по `2024-09-30`
- `pretrain_part_3.parquet` — 622.8MB, операции финальной трети клиентов с `2023-10-01` по `2024-09-30`
- `train_part_1.parquet` — 688.1MB, операции первой трети клиентов с `2024-10-01` по `2025-05-31`
- `train_part_2.parquet` — 686.5MB, операции второй трети клиентов с `2024-10-01` по `2025-05-31`
- `train_part_3.parquet` — 688.1MB, операции финальной трети клиентов с `2024-10-01` по `2025-05-31`

## Данные операций за Pre-test и Test

- `pretest.parquet` — 338.9MB, операции тестовых клиентов с `2025-06-01` по `2025-08-09`, предшествующие их последнему дню операций в данных
- `test.parquet` — 17.2MB, тестовые операции клиентов с `2025-06-01` по `2025-08-09`, за последний тестовый день каждого из тестовых клиентов

## Основные материалы соревнования

- `customer_id` — id клиента банка
- `event_id` — id операции
- `target` — целевая переменная, класс операции:
  - `1` соответствует неподтвержденной операции (🔴)
  - `0` соответствует напрямую подтвержденной клиентом операции (🟡)
  - Все остальные операции в данных считаются операциями без обратной связи (🟢)
- `train_labels.parquet` — разметка для тренировочных данных (Train)
- `sample_submit.csv` — пример базового решения

## Глоссарий данных об операциях клиентов

- `customer_id` — id клиента банка
- `event_id` — id операции
- `event_dttm` — дата/время операции
- `event_type_nm` — тип операции
- `event_desc` — закодированное описание операции
- `channel_indicator_type` — канал совершения операции
- `channel_indicator_subtype` — подтип канала совершения операции
- `operaton_amt` — сумма операции в рублях
- `currency_iso_cd` — валюта операции
- `mcc_code` — группа MCC merchant_category_code
- `pos_cd` — закодированное значение point of sale condition code
- `accept_language` — язык заголовка http запроса
- `browser_language` — язык браузера
- `timezone` — часовой пояс
- `session_id` — идентификатор сессии
- `operating_system_type` — закодированное значение операционной системы
- `battery` — заряд устройства
- `device_system_version` — версия операционной системы
- `screen_size` — разрешение экрана
- `developer_tools` — настройки разработчика (флаг на устройстве)
- `phone_voip_call_state` — флаг события VoIP-звонка во время проведения операции
- `web_rdp_connection` — флаг наличия удаленного управления над устройством
- `compromised` — наличие Root-доступа на устройстве

## 0) Environment setup (optional)

In [1]:
%pip install -U lightgbm polars pandas numpy pyarrow scikit-learn tqdm

Note: you may need to restart the kernel to use updated packages.


## 1) Imports and run configuration

In [2]:
from __future__ import annotations

import os
import math
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import polars as pl

import lightgbm as lgb
from sklearn.metrics import average_precision_score
from tqdm.auto import tqdm


@dataclass
class CFG:
    DATA_DIR_LOCAL: Path = Path(r"D:\Train Data\data fusion 2026")
    DATA_DIR_KAGGLE: Path = Path("/kaggle/input/fusion")  # Kaggle path: /kaggle/input/<dataset_name>
    DATA_DIR: Path = DATA_DIR_KAGGLE if DATA_DIR_KAGGLE.exists() else DATA_DIR_LOCAL

    OUT_DIR: Path = Path(os.environ.get("OUT_DIR", "./outputs"))

    PRETRAIN_FILES: List[str] = field(default_factory=lambda: [
        "pretrain_part_1.parquet",
        "pretrain_part_2.parquet",
        "pretrain_part_3.parquet",
    ])
    TRAIN_FILES: List[str] = field(default_factory=lambda: [
        "train_part_1.parquet",
        "train_part_2.parquet",
        "train_part_3.parquet",
    ])
    TRAIN_LABELS_FILE: str = "train_labels.parquet"
    PRETEST_FILE: str = "pretest.parquet"
    TEST_FILE: str = "test.parquet"

    CUSTOMER_COL: str = "customer_id"
    EVENT_ID_COL: str = "event_id"
    DT_COL: str = "event_dttm"
    TARGET_COL: str = "target"
    Y_COL: str = "y"

    GREEN_SAMPLE_FRAC: float = 0.02
    GREEN_MOD_BASE: int = 1000

    TARGET_ENCODER_COLS: Tuple[str, ...] = (
        "event_type_nm",
        "channel_indicator_type",
        "currency_iso_cd",
        "operating_system_type",
    )
    TARGET_ENCODER_ALPHA: float = 25.0

    COMBO_FREQ_COL_PAIRS: Tuple[Tuple[str, str], ...] = (
        ("event_type_nm", "channel_indicator_type"),
        ("event_type_nm", "currency_iso_cd"),
        ("channel_indicator_type", "operating_system_type"),
        ("event_type_nm", "pos_cd"),
    )
    COMBO_FREQ_MAX_UNIQUE: int = 120_000

    MAX_ROWS_PER_FILE: Optional[int] = None
    # --- split ---
    VALID_FROM: str = "2025-05-01"
    SEED: int = 42
    NUM_BOOST_ROUND: int = 2200
    EARLY_STOPPING: int = 120

    # --- optional: customer aggregates from pretrain ---
    BUILD_PRETRAIN_CUSTOMER_FEATS: bool = True
    PRETRAIN_CUST_FEATS_CACHE: str = "cust_feats_pretrain.parquet"


CFG = CFG()
max_rows_override = os.environ.get("MAX_ROWS_PER_FILE")
if max_rows_override:
    CFG.MAX_ROWS_PER_FILE = int(max_rows_override)
    print("MAX_ROWS_PER_FILE override:", CFG.MAX_ROWS_PER_FILE)

pretrain_override = os.environ.get("BUILD_PRETRAIN_CUSTOMER_FEATS")
if pretrain_override:
    CFG.BUILD_PRETRAIN_CUSTOMER_FEATS = pretrain_override.lower() not in {"0", "false", "no"}
    print("BUILD_PRETRAIN_CUSTOMER_FEATS override:", CFG.BUILD_PRETRAIN_CUSTOMER_FEATS)

CFG.OUT_DIR.mkdir(parents=True, exist_ok=True)
print("DATA_DIR =", CFG.DATA_DIR.resolve())
print("OUT_DIR  =", CFG.OUT_DIR.resolve())

DATA_DIR = D:\Train Data\data fusion 2026
OUT_DIR  = D:\Apps\PyCharm\pythonProject\fusion_ipynb\outputs


d:\Apps\PyCharm\pythonProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2) Utility functions for loading, feature prep, and encoding

In [3]:
def must_exist(fp: Path) -> None:
    if not fp.exists():
        raise FileNotFoundError(f"File not found: {fp}")


def pick_amount_col(cols: List[str]) -> Optional[str]:
    candidates = ["operaton_amt", "operation_amt", "operaton_amt_rub", "amount", "amt"]
    for c in candidates:
        if c in cols:
            return c
    return None


def select_existing(lf: pl.LazyFrame, cols: List[str]) -> pl.LazyFrame:
    schema = lf.collect_schema()
    existing = [c for c in cols if c in schema]
    return lf.select(existing)


def ensure_datetime(lf: pl.LazyFrame, col: str) -> pl.LazyFrame:
    schema = lf.collect_schema()
    if col not in schema:
        return lf
    if schema[col] == pl.Utf8:
        return lf.with_columns(pl.col(col).str.strptime(pl.Datetime, strict=False).alias(col))
    return lf


def add_time_features_pd(df: pd.DataFrame, dt_col: str) -> pd.DataFrame:
    dt = pd.to_datetime(df[dt_col], errors="coerce")
    df["hour"] = dt.dt.hour.astype("Int16")
    df["dow"] = dt.dt.dayofweek.astype("Int16")
    df["dom"] = dt.dt.day.astype("Int16")
    df["month"] = dt.dt.month.astype("Int16")
    df["is_weekend"] = (df["dow"] >= 5).astype("Int8")
    return df


def add_amount_features_pd(df: pd.DataFrame, amt_col: str) -> pd.DataFrame:
    amt = pd.to_numeric(df[amt_col], errors="coerce")
    df["amt"] = amt.astype("float32")
    df["amt_abs_log1p"] = np.log1p(np.abs(amt)).astype("float32")
    df["amt_sign"] = np.sign(amt.fillna(0)).astype("int8")
    return df


def infer_cat_cols(df: pd.DataFrame, exclude: set) -> List[str]:
    cat_cols: List[str] = []
    for c in df.columns:
        if c in exclude:
            continue
        dtype = df[c].dtype
        if (
            pd.api.types.is_object_dtype(dtype)
            or pd.api.types.is_string_dtype(dtype)
            or isinstance(dtype, pd.CategoricalDtype)
            or pd.api.types.is_bool_dtype(dtype)
        ):
            cat_cols.append(c)
    return cat_cols


def _as_string_series(s: pd.Series) -> pd.Series:
    return s.astype("string").fillna("__NA__")


def fit_freq_encoders(
    train_df: pd.DataFrame,
    cat_cols: List[str],
    *,
    max_unique: Optional[int] = 150_000,
) -> Dict[str, Dict[str, float]]:
    encoders: Dict[str, Dict[str, float]] = {}
    for c in tqdm(cat_cols, desc="Fitting freq encoders", leave=False):
        if c not in train_df.columns:
            continue
        s = _as_string_series(train_df[c])
        vc = s.value_counts(dropna=False)
        if max_unique is not None and len(vc) > max_unique:
            vc = vc.iloc[:max_unique]
        encoders[c] = {k: float(math.log1p(v)) for k, v in vc.items()}
    return encoders


def apply_freq_encoders(df: pd.DataFrame, encoders: Dict[str, Dict[str, float]]) -> pd.DataFrame:
    for c, mapping in tqdm(encoders.items(), desc="Applying freq encoders", leave=False):
        if c not in df.columns:
            continue
        s = _as_string_series(df[c])
        df[f"{c}__freq"] = s.map(mapping).fillna(0.0).astype("float32")
    return df


def fit_combo_freq_encoders(
    df: pd.DataFrame,
    col_pairs: List[Tuple[str, str]],
    *,
    max_unique: Optional[int] = 150_000,
) -> Dict[Tuple[str, str], Dict[Tuple[str, str], float]]:
    encoders: Dict[Tuple[str, str], Dict[Tuple[str, str], float]] = {}
    for c1, c2 in col_pairs:
        if c1 not in df.columns or c2 not in df.columns:
            continue
        s1 = _as_string_series(df[c1]).tolist()
        s2 = _as_string_series(df[c2]).tolist()
        combos = list(zip(s1, s2))
        if not combos:
            continue
        counts = pd.Series(combos).value_counts()
        if max_unique is not None and len(counts) > max_unique:
            counts = counts.iloc[:max_unique]
        encoders[(c1, c2)] = {
            pair: float(math.log1p(cnt)) for pair, cnt in counts.items()
        }
    return encoders


def apply_combo_freq_encoders(
    df: pd.DataFrame,
    combo_encoders: Dict[Tuple[str, str], Dict[Tuple[str, str], float]],
) -> pd.DataFrame:
    for (c1, c2), mapping in combo_encoders.items():
        if c1 not in df.columns or c2 not in df.columns:
            continue
        col_name = f"{c1}__{c2}__freq"
        s1 = _as_string_series(df[c1]).tolist()
        s2 = _as_string_series(df[c2]).tolist()
        df[col_name] = np.array(
            [mapping.get(pair, 0.0) for pair in zip(s1, s2)],
            dtype="float32",
        )
    return df


def fit_target_encoders(
    df: pd.DataFrame,
    cols: List[str],
    y_col: str,
    alpha: float = 25.0,
) -> Dict[str, Dict[str, float]]:
    encoders: Dict[str, Dict[str, float]] = {}
    if not cols:
        return encoders
    prior = float(df[y_col].mean())
    for col in cols:
        if col not in df.columns:
            continue
        temp = pd.DataFrame({col: _as_string_series(df[col]), y_col: df[y_col]})
        stats = temp.groupby(col)[y_col].agg(["mean", "count"])
        mapping: Dict[str, float] = {}
        for value, row in stats.iterrows():
            smooth = (row["mean"] * row["count"] + prior * alpha) / (row["count"] + alpha)
            mapping[value] = float(smooth)
        mapping["__prior__"] = prior
        encoders[col] = mapping
    return encoders


def apply_target_encoders(
    df: pd.DataFrame,
    target_encoders: Dict[str, Dict[str, float]],
) -> pd.DataFrame:
    for col, mapping in target_encoders.items():
        if col not in df.columns:
            continue
        prior = mapping.get("__prior__", 0.0)
        df[f"{col}__target_mean"] = (
            _as_string_series(df[col])
            .map(mapping)
            .fillna(prior)
            .astype("float32")
        )
    return df


def build_feature_matrix(
    df: pd.DataFrame,
    encoders: Dict[str, Dict[str, float]],
    amt_col: Optional[str],
    feat_cols: Optional[List[str]] = None,
    *,
    combo_encoders: Optional[Dict[Tuple[str, str], Dict[Tuple[str, str], float]]] = None,
    target_encoders: Optional[Dict[str, Dict[str, float]]] = None,
) -> Tuple[pd.DataFrame, List[str]]:
    df = df.copy()

    if CFG.DT_COL in df.columns:
        df = add_time_features_pd(df, CFG.DT_COL)

    if amt_col is not None and amt_col in df.columns:
        df = add_amount_features_pd(df, amt_col)
    else:
        df["amt"] = np.nan
        df["amt_abs_log1p"] = np.nan
        df["amt_sign"] = 0

    if encoders:
        df = apply_freq_encoders(df, encoders)
    if combo_encoders:
        df = apply_combo_freq_encoders(df, combo_encoders)
    if target_encoders:
        df = apply_target_encoders(df, target_encoders)

    drop_cols = {CFG.CUSTOMER_COL, CFG.EVENT_ID_COL, CFG.DT_COL, CFG.TARGET_COL, CFG.Y_COL}
    if feat_cols is None:
        feat_cols = []
        for c in df.columns:
            if c in drop_cols:
                continue
            if pd.api.types.is_numeric_dtype(df[c]):
                feat_cols.append(c)
    else:
        for c in feat_cols:
            if c not in df.columns:
                df[c] = np.nan

    X = df[feat_cols].replace([np.inf, -np.inf], np.nan).fillna(-1.0)
    return X, feat_cols


def build_customer_activity_stats(df: pd.DataFrame) -> pd.DataFrame:
    if CFG.CUSTOMER_COL not in df.columns or CFG.DT_COL not in df.columns:
        return pd.DataFrame()
    grouped = df.groupby(CFG.CUSTOMER_COL, observed=True)
    stats = grouped.agg(
        cust_total_ops=(CFG.EVENT_ID_COL, "count"),
        cust_unique_event_types=("event_type_nm", "nunique"),
        cust_unique_channels=("channel_indicator_type", "nunique"),
        cust_first_time=(CFG.DT_COL, "min"),
        cust_last_time=(CFG.DT_COL, "max"),
    )
    stats["cust_span_hours"] = (
        (stats["cust_last_time"] - stats["cust_first_time"]).dt.total_seconds().fillna(0.0) / 3600.0
    )
    stats["cust_span_hours"] = stats["cust_span_hours"].clip(lower=0.0)
    stats["cust_ops_per_hour"] = stats["cust_total_ops"] / stats["cust_span_hours"].replace(0.0, 1.0)
    stats["cust_ops_per_hour"] = stats["cust_ops_per_hour"].fillna(0.0)
    stats = stats.drop(columns=["cust_first_time", "cust_last_time"])
    return stats


def apply_customer_activity_stats(df: pd.DataFrame, stats: pd.DataFrame) -> pd.DataFrame:
    if stats.empty or CFG.CUSTOMER_COL not in df.columns:
        return df
    out = df.copy()
    for col in stats.columns:
        out[col] = out[CFG.CUSTOMER_COL].map(stats[col]).fillna(0.0).astype("float32")
    out["cust_total_ops_log"] = np.log1p(out["cust_total_ops"]).astype("float32")
    return out


def make_tqdm_callback(pbar: tqdm, postfix_every: int = 10):
    def _cb(env):
        pbar.update(1)
        if env.iteration % postfix_every == 0 and env.evaluation_result_list:
            valid_items = [x for x in env.evaluation_result_list if x[0] in ("valid", "val", "valid_0")]
            item = valid_items[0] if valid_items else env.evaluation_result_list[0]
            pbar.set_postfix({f"{item[0]}_{item[1]}": float(item[2])})
    _cb.order = 0
    return _cb

## 3) Input file checks and amount-column detection

In [4]:
# Check that all required files are available.
for name in CFG.PRETRAIN_FILES + CFG.TRAIN_FILES + [CFG.TRAIN_LABELS_FILE, CFG.PRETEST_FILE, CFG.TEST_FILE]:
    must_exist(CFG.DATA_DIR / name)

# Detect amount-like column from one TRAIN file schema.
train0 = pl.scan_parquet(CFG.DATA_DIR / CFG.TRAIN_FILES[0])
train0_schema = train0.collect_schema()
amt_col = pick_amount_col(list(train0_schema.keys()))
print("Detected amount column:", amt_col)

if amt_col is None:
    print("WARNING: No amount-like column was detected. Continuing without amount-derived features.")

Detected amount column: operaton_amt


## 4) Optional pre-train customer aggregates

In [5]:
def build_pretrain_customer_feats() -> Optional[pl.DataFrame]:
    if not CFG.BUILD_PRETRAIN_CUSTOMER_FEATS:
        return None

    cache_fp = CFG.OUT_DIR / CFG.PRETRAIN_CUST_FEATS_CACHE
    if cache_fp.exists():
        print("Loading cached customer feats:", cache_fp)
        return pl.read_parquet(cache_fp)

    print("Building customer feats from PRETRAIN (first run, will cache)...")


    base_cols = [CFG.CUSTOMER_COL]
    if amt_col is not None:
        base_cols.append(amt_col)

    maybe_cols = ["mcc_code", "channel_indicator_type", "event_type_nm", "currency_iso_cd"]
    use_cols = base_cols + maybe_cols

    lfs = []
    for name in tqdm(CFG.PRETRAIN_FILES, desc="Scanning pretrain parts"):
        lf = pl.scan_parquet(CFG.DATA_DIR / name)
        lf = select_existing(lf, use_cols)
        lfs.append(lf)

    lf_all = pl.concat(lfs, how="vertical_relaxed")

    aggs = [pl.len().alias("pt_cnt")]
    if amt_col is not None:
        aggs += [
            pl.col(amt_col).cast(pl.Float64).mean().alias("pt_amt_mean"),
            pl.col(amt_col).cast(pl.Float64).std().alias("pt_amt_std"),
            pl.col(amt_col).cast(pl.Float64).max().alias("pt_amt_max"),
        ]


    schema_all = lf_all.collect_schema()
    for c in maybe_cols:
        if c in schema_all:
            aggs.append(pl.col(c).n_unique().alias(f"pt_{c}_nuniq"))

    cust_feats = lf_all.group_by(CFG.CUSTOMER_COL).agg(aggs).collect(engine="streaming")
    cust_feats.write_parquet(cache_fp)
    print("Cached customer feats to:", cache_fp)

    return cust_feats

cust_feats_pl = build_pretrain_customer_feats()
print("cust_feats_pl:", None if cust_feats_pl is None else cust_feats_pl.shape)

Loading cached customer feats: outputs\cust_feats_pretrain.parquet
cust_feats_pl: (100000, 9)


## 5) Build sampled train frame with labels and customer features

In [6]:
def load_train_with_labels_sample() -> pl.DataFrame:
    labels_fp = CFG.DATA_DIR / CFG.TRAIN_LABELS_FILE
    labels_lf = pl.scan_parquet(labels_fp).select([CFG.EVENT_ID_COL, CFG.TARGET_COL])


    want_cols = [
        CFG.CUSTOMER_COL, CFG.EVENT_ID_COL, CFG.DT_COL,
        "event_type_nm", "event_desc",
        "channel_indicator_type", "channel_indicator_subtype",
        "currency_iso_cd", "mcc_code", "pos_cd",
        "accept_language", "browser_language", "timezone",
        "session_id", "operating_system_type", "device_system_version",
        "screen_size", "developer_tools", "phone_voip_call_state",
        "web_rdp_connection", "compromised", "battery",
    ]
    if amt_col is not None:
        want_cols.append(amt_col)

    parts: List[pl.DataFrame] = []
    thr = int(CFG.GREEN_SAMPLE_FRAC * CFG.GREEN_MOD_BASE)

    for name in tqdm(CFG.TRAIN_FILES, desc="Loading TRAIN parts"):
        fp = CFG.DATA_DIR / name
        lf = pl.scan_parquet(fp)

        if CFG.MAX_ROWS_PER_FILE is not None:
            lf = lf.head(CFG.MAX_ROWS_PER_FILE)

        lf = select_existing(lf, want_cols)
        lf = ensure_datetime(lf, CFG.DT_COL)

        lf = lf.join(labels_lf, on=CFG.EVENT_ID_COL, how="left")

        # Binary target: 1 for unconfirmed operations, 0 otherwise.
        lf = lf.with_columns((pl.col(CFG.TARGET_COL) == 1).cast(pl.Int8).fill_null(0).alias(CFG.Y_COL))

        is_labeled = pl.col(CFG.TARGET_COL).is_not_null()

        # Keep all labeled events.
        lf_labeled = lf.filter(is_labeled)



        lf_green = (
            lf.filter(~is_labeled)
              .with_columns(pl.col(CFG.EVENT_ID_COL).cast(pl.Int64))
              .filter((pl.col(CFG.EVENT_ID_COL) % CFG.GREEN_MOD_BASE) < thr)
        )

        lf_out = pl.concat([lf_labeled, lf_green], how="vertical_relaxed")


        if cust_feats_pl is not None:
            lf_out = lf_out.join(cust_feats_pl.lazy(), on=CFG.CUSTOMER_COL, how="left")

        parts.append(lf_out.collect(engine="streaming"))

    return pl.concat(parts, how="vertical_relaxed")

train_pl = load_train_with_labels_sample()
print("Train sampled shape:", train_pl.shape)

train_df = train_pl.to_pandas(use_pyarrow_extension_array=False)
del train_pl


train_df[CFG.DT_COL] = pd.to_datetime(train_df[CFG.DT_COL], errors="coerce")

print("y counts:", train_df[CFG.Y_COL].value_counts(dropna=False).to_dict())
print("pos rate:", float(train_df[CFG.Y_COL].mean()))

CUSTOMER_ACTIVITY_STATS = build_customer_activity_stats(train_df)
train_df = apply_customer_activity_stats(train_df, CUSTOMER_ACTIVITY_STATS)

Loading TRAIN parts: 100%|██████████| 3/3 [00:44<00:00, 14.91s/it]


Train sampled shape: (1799067, 32)
y counts: {0: 1747629, 1: 51438}
pos rate: 0.028591486587214374


## 6) Time-based split, feature encoding, and blend search

In [7]:
from itertools import combinations

valid_from_ts = pd.Timestamp(CFG.VALID_FROM)
is_valid = train_df[CFG.DT_COL] >= valid_from_ts

train_part = train_df.loc[~is_valid].copy()
valid_part = train_df.loc[is_valid].copy()

print("Train part:", train_part.shape, "Valid part:", valid_part.shape)
print("Pos rate train:", float(train_part[CFG.Y_COL].mean()), "Pos rate valid:", float(valid_part[CFG.Y_COL].mean()))

exclude = {CFG.CUSTOMER_COL, CFG.EVENT_ID_COL, CFG.DT_COL, CFG.TARGET_COL, CFG.Y_COL}
cat_cols = infer_cat_cols(train_part, exclude=exclude)
print("Detected categorical cols:", len(cat_cols))
print("Example:", cat_cols[:20])

encoders = fit_freq_encoders(train_part, cat_cols, max_unique=150_000)
combo_encoders_cv = fit_combo_freq_encoders(
    train_part,
    CFG.COMBO_FREQ_COL_PAIRS,
    max_unique=CFG.COMBO_FREQ_MAX_UNIQUE,
)
target_encoders_cv = fit_target_encoders(
    train_part,
    CFG.TARGET_ENCODER_COLS,
    CFG.Y_COL,
    CFG.TARGET_ENCODER_ALPHA,
)

X_train, feat_cols = build_feature_matrix(
    train_part,
    encoders,
    amt_col=amt_col,
    combo_encoders=combo_encoders_cv,
    target_encoders=target_encoders_cv,
)
y_train = train_part[CFG.Y_COL].astype("int8").to_numpy()

X_valid, _ = build_feature_matrix(
    valid_part,
    encoders,
    amt_col=amt_col,
    feat_cols=feat_cols,
    combo_encoders=combo_encoders_cv,
    target_encoders=target_encoders_cv,
)
y_valid = valid_part[CFG.Y_COL].astype("int8").to_numpy()

pos = float(np.sum(y_train == 1))
neg = float(np.sum(y_train == 0))
scale_pos_weight = neg / max(pos, 1.0)
print("scale_pos_weight =", scale_pos_weight)

MODEL_SPECS = [
    {
        "name": "m0",
        "seed": 42,
        "params": {
            "learning_rate": 0.05,
            "num_leaves": 64,
            "min_data_in_leaf": 200,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
            "lambda_l2": 1.0,
        },
    },
    {
        "name": "m1",
        "seed": 73,
        "params": {
            "learning_rate": 0.04,
            "num_leaves": 96,
            "min_data_in_leaf": 150,
            "feature_fraction": 0.85,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
            "lambda_l2": 2.0,
        },
    },
    {
        "name": "tuned_b",
        "seed": 73,
        "params": {
            "learning_rate": 0.03,
            "num_leaves": 128,
            "min_data_in_leaf": 120,
            "feature_fraction": 0.9,
            "bagging_fraction": 0.85,
            "bagging_freq": 1,
            "lambda_l2": 2.0,
        },
    },
    {
        "name": "tuned_d",
        "seed": 42,
        "params": {
            "learning_rate": 0.04,
            "num_leaves": 80,
            "min_data_in_leaf": 300,
            "feature_fraction": 0.8,
            "bagging_fraction": 0.8,
            "bagging_freq": 1,
            "lambda_l2": 3.0,
        },
    },
    {
        "name": "tuned_e",
        "seed": 73,
        "params": {
            "learning_rate": 0.045,
            "num_leaves": 96,
            "min_data_in_leaf": 220,
            "feature_fraction": 0.88,
            "bagging_fraction": 0.9,
            "bagging_freq": 1,
            "lambda_l2": 1.7,
        },
    },
]

split_models = []
for spec in MODEL_SPECS:
    print(f"\n--- Train split model: {spec['name']} (seed={spec['seed']}) ---")

    params_local = {
        "objective": "binary",
        "metric": ["average_precision"],
        "max_bin": 255,
        "verbosity": -1,
        "seed": spec["seed"],
        "scale_pos_weight": scale_pos_weight,
        "num_threads": os.cpu_count() or 4,
    }
    params_local.update(spec["params"])

    lgb_train = lgb.Dataset(X_train, label=y_train, feature_name=feat_cols, free_raw_data=True)
    lgb_valid = lgb.Dataset(X_valid, label=y_valid, feature_name=feat_cols, reference=lgb_train, free_raw_data=True)

    model_local = lgb.train(
        params=params_local,
        train_set=lgb_train,
        num_boost_round=CFG.NUM_BOOST_ROUND,
        valid_sets=[lgb_valid],
        valid_names=["valid"],
        callbacks=[
            lgb.early_stopping(CFG.EARLY_STOPPING, verbose=False),
            lgb.log_evaluation(period=0),
        ],
    )

    best_iter_local = int(model_local.best_iteration or CFG.NUM_BOOST_ROUND)
    valid_pred_local = model_local.predict(X_valid, num_iteration=best_iter_local)
    valid_ap_local = float(average_precision_score(y_valid, valid_pred_local))

    print(f"{spec['name']}: best_iter={best_iter_local}, valid_ap={valid_ap_local:.6f}")

    split_models.append(
        {
            "name": spec["name"],
            "seed": spec["seed"],
            "params": params_local,
            "best_iter": best_iter_local,
            "valid_ap": valid_ap_local,
            "valid_pred": valid_pred_local,
        }
    )

# Search best pair blend on validation
best_blend = {
    "ap": -1.0,
    "kind": None,
    "members": [],
    "weights": [],
}
name_to_pred = {m["name"]: m["valid_pred"] for m in split_models}
model_names = [m["name"] for m in split_models]

for a, b in combinations(model_names, 2):
    p_a = name_to_pred[a]
    p_b = name_to_pred[b]
    for w in np.linspace(0.0, 1.0, 41):
        p_bl = (1.0 - w) * p_a + w * p_b
        ap_bl = float(average_precision_score(y_valid, p_bl))
        if ap_bl > best_blend["ap"]:
            best_blend = {
                "ap": ap_bl,
                "kind": "pair",
                "members": [a, b],
                "weights": [1.0 - float(w), float(w)],
            }

# Optional top-3 triple blend (coarse simplex)
top3 = sorted(split_models, key=lambda d: d["valid_ap"], reverse=True)[:3]
if len(top3) == 3:
    n0, n1, n2 = [x["name"] for x in top3]
    p0, p1, p2 = [name_to_pred[n0], name_to_pred[n1], name_to_pred[n2]]
    grid = np.linspace(0.0, 1.0, 11)
    for w0 in grid:
        for w1 in grid:
            w2 = 1.0 - w0 - w1
            if w2 < 0:
                continue
            p_bl = w0 * p0 + w1 * p1 + w2 * p2
            ap_bl = float(average_precision_score(y_valid, p_bl))
            if ap_bl > best_blend["ap"]:
                best_blend = {
                    "ap": ap_bl,
                    "kind": "triple",
                    "members": [n0, n1, n2],
                    "weights": [float(w0), float(w1), float(w2)],
                }

print(
    "Best validation blend:",
    f"kind={best_blend['kind']}",
    f"members={best_blend['members']}",
    f"weights={[round(w, 4) for w in best_blend['weights']]}",
    f"valid_ap={best_blend['ap']:.6f}",
)

combo_encoders_full = fit_combo_freq_encoders(
    train_df,
    CFG.COMBO_FREQ_COL_PAIRS,
    max_unique=CFG.COMBO_FREQ_MAX_UNIQUE,
)
target_encoders_full = fit_target_encoders(
    train_df,
    CFG.TARGET_ENCODER_COLS,
    CFG.Y_COL,
    CFG.TARGET_ENCODER_ALPHA,
)

# Refit selected models on full sampled train
name_to_split = {m["name"]: m for m in split_models}

X_full, _ = build_feature_matrix(
    train_df,
    encoders,
    amt_col=amt_col,
    feat_cols=feat_cols,
    combo_encoders=combo_encoders_full,
    target_encoders=target_encoders_full,
)
y_full = train_df[CFG.Y_COL].astype("int8").to_numpy()

pos_full = float(np.sum(y_full == 1))
neg_full = float(np.sum(y_full == 0))
scale_pos_weight_full = neg_full / max(pos_full, 1.0)

ENSEMBLE_MODELS = []
for name in best_blend["members"]:
    sm = name_to_split[name]
    print(f"Refit full: {name}")

    params_full = dict(sm["params"])
    params_full["scale_pos_weight"] = scale_pos_weight_full

    lgb_full = lgb.Dataset(X_full, label=y_full, feature_name=feat_cols, free_raw_data=True)
    model_full = lgb.train(
        params=params_full,
        train_set=lgb_full,
        num_boost_round=sm["best_iter"],
        valid_sets=[],
        callbacks=[lgb.log_evaluation(period=0)],
    )

    ENSEMBLE_MODELS.append(
        {
            "name": name,
            "model": model_full,
            "best_iter": sm["best_iter"],
        }
    )

ENSEMBLE_WEIGHTS = np.asarray(best_blend["weights"], dtype="float64")
print("Final ensemble members:", [m["name"] for m in ENSEMBLE_MODELS])
print("Final ensemble weights:", ENSEMBLE_WEIGHTS.tolist())

Train part: (1534614, 38) Valid part: (264453, 38)
Pos rate train: 0.02895907374753521 Pos rate valid: 0.026458387690818406
Detected categorical cols: 8
Example: ['mcc_code', 'accept_language', 'browser_language', 'device_system_version', 'screen_size', 'developer_tools', 'compromised', 'battery']


scale_pos_weight = 33.531491190567266

--- Train split model: m0 (seed=42) ---
m0: best_iter=282, valid_ap=0.248375

--- Train split model: m1 (seed=73) ---
m1: best_iter=331, valid_ap=0.247618

--- Train split model: tuned_b (seed=73) ---
tuned_b: best_iter=429, valid_ap=0.248640

--- Train split model: tuned_d (seed=42) ---
tuned_d: best_iter=380, valid_ap=0.252128

--- Train split model: tuned_e (seed=73) ---
tuned_e: best_iter=309, valid_ap=0.248978
Best validation blend: kind=triple members=['tuned_d', 'tuned_e', 'tuned_b'] weights=[0.7, 0.2, 0.1] valid_ap=0.252645


Refit full: tuned_d
Refit full: tuned_e
Refit full: tuned_b
Final ensemble members: ['tuned_d', 'tuned_e', 'tuned_b']
Final ensemble weights: [0.7000000000000001, 0.2, 0.09999999999999992]


## 7) Fit final ensemble and generate test submission

In [8]:
test_fp = CFG.DATA_DIR / CFG.TEST_FILE
test_pl = pl.read_parquet(test_fp)

if cust_feats_pl is not None:
    test_pl = test_pl.join(cust_feats_pl, on=CFG.CUSTOMER_COL, how="left")

test_df = test_pl.to_pandas(use_pyarrow_extension_array=False)
del test_pl

if CFG.DT_COL in test_df.columns:
    test_df[CFG.DT_COL] = pd.to_datetime(test_df[CFG.DT_COL], errors="coerce")

test_df = apply_customer_activity_stats(test_df, CUSTOMER_ACTIVITY_STATS)

X_test, _ = build_feature_matrix(
    test_df,
    encoders,
    amt_col=amt_col,
    feat_cols=feat_cols,
    combo_encoders=combo_encoders_full,
    target_encoders=target_encoders_full,
)

pred_parts = []
for m in ENSEMBLE_MODELS:
    p_m = m["model"].predict(X_test, num_iteration=m["best_iter"])
    pred_parts.append(p_m)
    print(f"{m['name']} pred stats:", float(np.min(p_m)), float(np.mean(p_m)), float(np.max(p_m)))

pred = np.zeros(len(test_df), dtype="float64")
for w, p_m in zip(ENSEMBLE_WEIGHTS, pred_parts):
    pred += float(w) * p_m

submit_path = CFG.OUT_DIR / "8.csv"
out = pd.DataFrame({CFG.EVENT_ID_COL: test_df[CFG.EVENT_ID_COL].values, "predict": pred})
out.to_csv(submit_path, index=False)

print("Submission saved to:", submit_path.resolve())
print("Rows:", len(out))

tuned_d pred stats: 0.001160679798217419 0.29632596341369616 0.9912954566434496
tuned_e pred stats: 0.0010949394444196604 0.2913249663682736 0.9912875647342297
tuned_b pred stats: 0.0009310635691051099 0.2844309022556963 0.9897430167280525
Submission saved to: D:\Apps\PyCharm\pythonProject\fusion_ipynb\outputs\8.csv
Rows: 633683


## 8) Additional honest-ish validation snapshot

In [9]:
import polars as pl
import numpy as np
from sklearn.metrics import average_precision_score

VALID_FROM = CFG.VALID_FROM
VALID_TO = "2025-05-31"
GREEN_PER_LABELED = 30

labels_lf = (
    pl.scan_parquet(CFG.DATA_DIR / CFG.TRAIN_LABELS_FILE)
      .select([CFG.EVENT_ID_COL, CFG.TARGET_COL])
)

valid_parts = []
for name in CFG.TRAIN_FILES:
    ops = pl.scan_parquet(CFG.DATA_DIR / name)
    ops = ensure_datetime(ops, CFG.DT_COL)

    ops_win = ops.filter(
        (pl.col(CFG.DT_COL) >= pl.datetime(*map(int, VALID_FROM.split("-")), 0, 0, 0))
        & (pl.col(CFG.DT_COL) <= pl.datetime(*map(int, VALID_TO.split("-")), 23, 59, 59))
    )

    joined = ops_win.join(labels_lf, on=CFG.EVENT_ID_COL, how="left")

    labeled = joined.filter(pl.col(CFG.TARGET_COL).is_not_null()).with_columns(
        (pl.col(CFG.TARGET_COL) == 1).cast(pl.Int8).alias("y")
    )

    green = joined.filter(pl.col(CFG.TARGET_COL).is_null()).with_columns(
        pl.lit(0).cast(pl.Int8).alias("y")
    )

    labeled_df = labeled.collect(engine="streaming")
    green_df = green.collect(engine="streaming")
    want_green = int(labeled_df.height * GREEN_PER_LABELED)

    green_s = (
        green_df.sample(n=min(want_green, green_df.height), seed=CFG.SEED)
        if want_green > 0
        else green_df.head(0)
    )

    part = pl.concat([labeled_df, green_s], how="vertical_relaxed")
    if cust_feats_pl is not None:
        part = part.join(cust_feats_pl, on=CFG.CUSTOMER_COL, how="left")

    valid_parts.append(part)

valid_df = pl.concat(valid_parts, how="vertical_relaxed").to_pandas(use_pyarrow_extension_array=False)

valid_df[CFG.DT_COL] = pd.to_datetime(valid_df[CFG.DT_COL], errors="coerce")
y_valid_honest = valid_df["y"].astype("int8").to_numpy()

valid_df = apply_customer_activity_stats(valid_df, CUSTOMER_ACTIVITY_STATS)

X_valid_honest, _ = build_feature_matrix(
    valid_df,
    encoders,
    amt_col=amt_col,
    feat_cols=feat_cols,
    combo_encoders=combo_encoders_cv,
    target_encoders=target_encoders_cv,
)

pred_parts = []
for m in ENSEMBLE_MODELS:
    p_m = m["model"].predict(X_valid_honest, num_iteration=m["best_iter"])
    ap_m = float(average_precision_score(y_valid_honest, p_m))
    print(f"honest-ish AP ({m['name']}):", ap_m)
    pred_parts.append(p_m)

p_valid_honest = np.zeros(len(valid_df), dtype="float64")
for w, p_m in zip(ENSEMBLE_WEIGHTS, pred_parts):
    p_valid_honest += float(w) * p_m

print("honest-ish valid pos_rate =", float(y_valid_honest.mean()))
print("honest-ish PR-AUC (AP)    =", float(average_precision_score(y_valid_honest, p_valid_honest)))

honest-ish AP (tuned_d): 0.29277600301119544
honest-ish AP (tuned_e): 0.3009941490866072
honest-ish AP (tuned_b): 0.3220252269541291
honest-ish valid pos_rate = 0.01680637955468018
honest-ish PR-AUC (AP)    = 0.3009431598066904


In [11]:
'''
Final PR-AUC on public ds = 0,1398083541
At the time of submitting the solution it was 2nd place on leaderboard
'''

'\nFinal PR-AUC on public ds = 0,1398083541\nAt the time of submitting the solution it was 2nd place on leaderboard\n'